In [ ]:
import json
import boto3
import time

session = boto3.Session(aws_access_key_id='Your_Access_Key_id',aws_secret_access_key='Your_Secret_Access_Key')
temperatureClient = session.client('firehose',region_name='us-east-1')


with open('tweets.json','r',encoding='utf-8') as json_file:
    observations = json.loads(json_file.read())

    for observation in range(0,14000):
        # I removed the tweet_coord because crawler can't understand its structure
        del observations[observation]['tweet_coord']
        
#         print(observations[observation])
#         if observation % 20 == 0:
#             time.sleep(1)
        print(observations[observation])
        response = temperatureClient.put_record(
           DeliveryStreamName='PUT-S3-HLnli',
           Record={
                'Data': json.dumps(observations[observation])+'\n'
            }
        )
        print(response)

In [ ]:
bucket_name = 'row-data'

session = boto3.Session(aws_access_key_id='Your_Access_Key_id',aws_secret_access_key='Your_Secret_Access_Key')
s3 = session.client('s3')
kinises = session.client('firehose',region_name='us-east-1')

keys = s3.list_objects_v2(Bucket=bucket_name)['Contents']
for key in keys:
    file = s3.get_object(Bucket = bucket_name, Key =key['Key'])

    pragraphs = str(file['Body'].read().decode("utf-8"))[:-1]
    pragraphs = pragraphs.split("\n")
    
    
    for pragraph in pragraphs:
        pragraph = json.loads(pragraph)
        
        Text = pragraph['text']
        #pass to comperhend and get the response
        comperhand = session.client('comprehend',region_name='us-east-1')
        response = comperhand.detect_sentiment(Text = Text, LanguageCode = 'en')
        # add it to the record
        pragraph['Sentiment'] = response['Sentiment']

        response = kinises.put_record(
                   DeliveryStreamName='PUT-S3-for-athena',
                   Record={
                        'Data': json.dumps(pragraph)+'\n'
                    }
        )
    s3.delete_object(Bucket=bucket_name, Key=key['Key'])